In [1]:
import sys
sys.path.append('../_amigocloud')

from amigocloud import AmigoCloud

from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections

from datetime import datetime

from shapely import wkb
import geopandas
import pandas as pd
import contextily as ctx
import matplotlib.pyplot as plt

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)

In [2]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [3]:
def generar_reporte(codigo_canhero):
    # EXTRAER DATOS DE CREDITO
    query_credito = {'query': 'select * from dataset_307560 where cast(SPLIT_PART(canhero, \'/\', 1) as integer) = {cod_ca}'.format(cod_ca=codigo_canhero)}
    credito_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_credito)
    credito_data = credito_select['data'][0]
    credito_data['fecha_informe'] = datetime.strptime(credito_data['fecha_3ra_insp'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')
    credito_data['fecha_3ra_insp'] = datetime.strptime(credito_data['fecha_3ra_insp'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')
    credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())

    # EXTRAER DATOS DE PROPIEDADES
    # query_propiedades = {'query': 'select * from dataset_282968 where cre_ref_prop = \'{cre_ref_prop}\''.format(cre_ref_prop=credito.amigo_id)}
    query_propiedades = {'query': 'select prop.propiedad, prop.amigo_id from dataset_307562 prop inner join dataset_307563 lotes on lotes.prop_ref_lote=prop.amigo_id where insp_ref_prop=\'{cre_ref_prop}\' and (lotes.estado_de_siembra=\'CAÑA NACIDA\' or lotes.estado_de_siembra=\'CAÑA NO NACIDA\') group by prop.propiedad, prop.amigo_id'.format(cre_ref_prop=credito.amigo_id)}
    propiedades_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_propiedades)    
    
    propiedades_data = propiedades_select['data']

    # EXTRAER DATOS DE LOTES
    propiedades = []
    for prop in propiedades_data:
        prop['lotes'] = []
        query_lotes = {'query': 'select * from dataset_307563 where prop_ref_lote=\'{prop_ref_lote}\' and (estado_de_siembra=\'CAÑA NACIDA\' or estado_de_siembra=\'CAÑA NO NACIDA\')'.format(prop_ref_lote=prop['amigo_id'])}
        lotes_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_lotes)
        for lote_select in lotes_select['data']:
            lote = collections.namedtuple("lote", lote_select.keys())(*lote_select.values())
            prop['lotes'].append(lote)
        propiedades.append(collections.namedtuple("propiedad", prop.keys())(*prop.values()))

    # EXTRAER FOTOS
    fotos = []
    for propiedad in propiedades:
        for lote in propiedad.lotes:
            print(lote.lote, lote.amigo_id)
            query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_58438 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=lote.amigo_id)}
            fotos_filename = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_fotos)
            if fotos_filename['data'] != []:
                dic = {'lote': lote.lote, 'file_name': fotos_filename['data'][0]['s3_filename']}
                fotos.append(dic)

    # GENERERAR PLANOS
    i = 0
    lista_planos = []
    for propiedad in propiedades:
        i = i + 1
        lista_planos.append(generar_plano(propiedad, i))


    # GENERAR REPORTE
    doc = DocxTemplate(ruta + "/templates/tpl_inf_credito_3ra.docx")
    #generar lista de InlineImage de planos 
    lista_InlineImage = []
    for plano in lista_planos:
        lista_InlineImage.append(docxtpl.InlineImage(doc, image_descriptor=plano, width=Mm(150)))
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        file = open(ruta + '/fotos/' + foto['file_name'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor=ruta + '/fotos/' + foto['file_name'], width=Mm(120)), 'lote': foto['lote']})
    firma_respon = None
    if credito.respon_2da_insp == 'Juan Pablo Jaldin Duran':
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta + '/templates/firma_jaldin.png', width=Mm(60))
    else:
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta + '/templates/firma_rogelio.png', width=Mm(60))
    context = {'credito':credito, 'propiedades':propiedades, 'planos':lista_InlineImage, 'fotos':lista_fotos_inline, 'firma':firma_respon}
    doc.render(context)
    cod_ca = credito.canhero.split(' / ')[0]
    nom_ca = credito.canhero.split(' / ')[1]
    file_name = file_name = '_' + cod_ca + '_CRS_' + credito.fecha_3ra_insp.replace('/', '-') + '_' + nom_ca + '_' + str(credito.id)
    doc.save(ruta + '/informes/' + file_name + '.docx')

In [4]:
def generar_plano(propiedad, i):
    lotes_lista = []
    for lote in propiedad.lotes:
        lotes_lista.append(lote._asdict())
    df = pd.DataFrame(lotes_lista)
    df['geometria'] = df['geometria'].apply(convertir_wkb)

    #Convertir a GeoDataFrame
    data = geopandas.GeoDataFrame(df, geometry='geometria')
    #data
    #ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)

    data['coords'] = data['geometria'].apply(lambda x: x.representative_point().coords[:])
    data['coords'] = [coords[0] for coords in data['coords']]
    
    
    data.crs = "EPSG:4326"
    data = data.to_crs(epsg=3857)
    print(data.crs.name)
    
    fig = plt.figure(i, figsize=(20,20))
    #fig = plt.figure(1)
    ax = None
    ax = fig.add_subplot()

    #data.set_crs(epsg=4326, inplace=True)

    data.apply(lambda x: ax.annotate(text=x.lote + ' \n' + str(x.superficie) + ' ha', xy=x.geometria.centroid.coords[0], ha='center', va='center', color='black', fontsize=12, weight=1000, bbox=dict(facecolor=(1,1,1,0.3), edgecolor='none', pad=0)), axis=1);

    #data.boundary.plot(ax=ax, color=(0,0,0,0), linewidth=0)

    #data = data.to_crs(epsg=3857)

    minx, miny, maxx, maxy = data.total_bounds
    ax.set_xlim(minx - 500, maxx + 500)
    ax.set_ylim(miny - 400, maxy + 400)

    data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2, figsize=(20,20))
    #data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2)
    
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)
    ax.set_axis_off()
    ax.set_title(str(propiedad.propiedad), fontsize=20)
    ruta_plano = ruta + '/planos/' + str(propiedad.amigo_id) + '.jpeg'
    fig.savefig(ruta_plano, dpi = 300, bbox_inches='tight')
    plt.clf()
    return ruta_plano

In [6]:
semana = 'S27'
query = {'query': 'select split_part(canhero, \' / \', 1) cod, split_part(canhero, \' / \', 2) nom from dataset_307560 where semana_3ra = \'{sem}\''.format(sem=semana)}
project = 'https://app.amigocloud.com/api/v1/projects/32672/sql'
select = amigocloud.get(project, query)
data = select['data']

In [7]:
data

[{'cod': '41599', 'nom': 'LOPEZ TURIHUANO JUAN CARLOS'},
 {'cod': '1237', 'nom': 'BALCAZAR GUTIERREZ JOSE IVER'},
 {'cod': '5001', 'nom': 'GUZMAN CACERES ROXANA'},
 {'cod': '40158', 'nom': 'GUZMAN CACERES MARIA LOURDES'},
 {'cod': '13062', 'nom': 'SOTO ARCE BACILIO'},
 {'cod': '8684', 'nom': 'MOLINA PORCEL JUAN CARLOS'},
 {'cod': '41529', 'nom': 'ACUÑA CANO VICTORIANO'},
 {'cod': '42239', 'nom': 'CORDOVA OLGUIN LEONARDO'},
 {'cod': '12162', 'nom': 'ROCA REYES MARCOS'}]

In [8]:
# lista = [430, 40270, 13132, 874, 41966, 11798, 41544, 14993, 644, 1068, 13763, 2220, 1858, 2719]
lista = [i['cod'] for i in data]

In [9]:
lista

['41599', '1237', '5001', '40158', '13062', '8684', '41529', '42239', '12162']

In [10]:
for i in lista:
    generar_reporte(i)
    print('======================', i)

L1 8d35205acdd34999a48ca66175830088
L3 d324460557b64ac09194bd5db8d2bf6a
L2 367286f9a6e742ba93343da4eb7212f6
L3.1 433384df55a741288b4df658d43dda5a
L1 884916c4cf4b4117af36868c06f86cd8
L4 1cf1d5dd2dfb40c6ae7329419810bcb0
L3.1 1972533fce3b405187ccbc8357bbcf96
WGS 84 / Pseudo-Mercator
WGS 84 / Pseudo-Mercator
WGS 84 / Pseudo-Mercator
WGS 84 / Pseudo-Mercator
====================== 41599
L7 01029e4739a848cc9a88f6a4b376b0e9
L1 83e8ce3dfc1249f68fa60430f2764ed7
L5 d59c62a7051f4c15816ad5481fa0d02e
L6 6d114ba541e34aac8054aa770c4bb637
L2 cf4719d90ac84c0499088e094a175b1a
L3 0f9b32df465c48cc8c3c85c0b066a2cf
L4 97f59f39944844f5b7714a7d6804b71a
WGS 84 / Pseudo-Mercator
====================== 1237
L5 9b3977ad94e845cc911ac8690552cc91
L2.2 8a1fbccf00b1443e966926f42c062d16
L2.1 6676520e18704cf98453b321bcddf0bc
WGS 84 / Pseudo-Mercator
====================== 5001
L15 1a76caf74fab4b8b833a2f12c17f764c
L20.1 0ac32bdecb9e4a4fab4ae511544ac357
WGS 84 / Pseudo-Mercator
====================== 40158
L4.1 674ce1bf50

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>